## Starter Code for Assignment 2 - Big Data with H2O

**Binary Classification**

Here is some starter code for Assignment 2 - Big Data with H2O.   You will still need to extend this code somewhat.  


[H2O Docs](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html)  
[http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html)  

In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool

Set up some parameters for the analysis.  

In [2]:
data_path=None
all_variables=None
test_path=None
target=None
nthreads=1 
min_mem_size=6 
run_time=333
classification=False
scale=False
max_models=9    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0 

The next sections contains helper functions for automating analysis.             

In [3]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(run_id,analysis,target,run_time,classification,scale,model,balance,balance_threshold,name,nthreads,min_mem_size):
  m_data={}
  m_data['run_id'] =run_id
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [4]:
data_path='data/twoclass_loan.csv'

In [5]:
data_path = os.path.join(os.path.abspath(os.curdir),data_path)

In [6]:
all_variables=None

In [7]:
run_id=alphabet(9)
# run_id to std out
print (run_id) 

hunAq3K6I


In [8]:
server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir) 

In [9]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:45444..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /Users/bear/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpgicv38qo
  JVM stdout: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpgicv38qo/h2o_bear_started_from_python.out
  JVM stderr: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpgicv38qo/h2o_bear_started_from_python.err
  Server is running at http://127.0.0.1:45444
Connecting to H2O server at http://127.0.0.1:45444... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.1
H2O cluster version age:,3 months and 25 days !!!
H2O cluster name:,H2O_from_python_bear_ywvjvo
H2O cluster total nodes:,1
H2O cluster free memory:,5.750 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [10]:
# meta data
meta_data = set_meta_data(run_id,analysis,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,nthreads,min_mem_size)
print(meta_data)  

{'run_id': 'hunAq3K6I', 'start_time': 1538541163.6288798, 'target': None, 'max_models': 9, 'run_time': 333, 'scale': False, 'classification': False, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1538541163.628885, 'execution_time': 0.0, 'nthreads': 1, 'min_mem_size': 6, 'analysis': 0}


In [11]:
print(data_path)

/Users/bear/Downloads/AutoML/data/twoclass_loan.csv


In [12]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
df.head()

loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
5000,36 months,10.65,10,RENT,24000,credit_card,AZ,27.65,0,83.7,9,0,26,verified
2500,60 months,15.27,0,RENT,30000,car,GA,1,0,9.4,4,1,12,verified
2400,36 months,15.96,10,RENT,12252,small_business,IL,8.72,0,98.5,10,0,10,not verified
10000,36 months,13.49,10,RENT,49200,other,CA,20,0,21,37,0,15,verified
5000,36 months,7.9,3,RENT,36000,wedding,AZ,11.2,0,28.3,12,0,7,verified
3000,36 months,18.64,9,RENT,48000,car,CA,5.35,0,87.5,4,0,4,verified
5600,60 months,21.28,4,OWN,40000,small_business,CA,5.55,0,32.6,13,1,7,verified
5375,60 months,12.69,0,RENT,15000,other,TX,18.08,0,36.5,3,1,7,verified
6500,60 months,14.65,5,OWN,72000,debt_consolidation,AZ,16.12,0,20.6,23,0,13,not verified
12000,36 months,12.69,10,OWN,75000,debt_consolidation,CA,10.78,0,67.1,34,0,22,verified


In [14]:
df.describe()

Rows:163987
Cols:15




,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
type,int,enum,real,int,enum,real,enum,enum,real,int,real,int,int,int,enum
mins,500.0,,5.42,0.0,,1896.0,,,0.0,0.0,0.0,1.0,0.0,0.0,
mean,13074.169141456336,,13.715904065566173,5.68435293299533,,71915.67051974901,,,15.881530121290117,0.2273570060625282,54.07917280242258,24.579733834274638,0.1830388994249544,14.854273655448353,
maxs,35000.0,,26.06,10.0,,7141778.0,,,39.99,29.0,150.70000000000002,118.0,1.0,65.0,
sigma,7993.556188734649,,4.391939870545795,3.6106637311002365,,59070.915654918244,,,7.587668224192549,0.6941679229284182,25.285366766770505,11.685190365910659,0.3866995896078875,6.947732922546696,
zeros,0,,0,14248,,0,,,270,139459,1562,0,133971,11,
missing,0,0,0,5804,0,4,0,0,0,29,193,29,0,29,0
0,5000.0,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0.0,26.0,verified
1,2500.0,60 months,15.27,0.0,RENT,30000.0,car,GA,1.0,0.0,9.4,4.0,1.0,12.0,verified
2,2400.0,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0.0,10.0,not verified


describe()[source]
Generate an in-depth description of this H2OFrame.

The description is a tabular print of the type, min, max, sigma, number of zeros, and number of missing elements for each H2OVec in this H2OFrame.

Returns:	None (print to stdout) 

In [15]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
  target=df.columns[-1]   
y = target

In [16]:
print(y)

verification_status


In [17]:
print(all_variables)

None


In [18]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [19]:
df.describe()

Rows:163987
Cols:15




,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
type,int,enum,real,int,enum,real,enum,enum,real,int,real,int,int,int,enum
mins,500.0,,5.42,0.0,,1896.0,,,0.0,0.0,0.0,1.0,0.0,0.0,
mean,13074.169141456336,,13.715904065566173,5.68435293299533,,71915.67051974901,,,15.881530121290117,0.2273570060625282,54.07917280242258,24.579733834274638,0.1830388994249544,14.854273655448353,
maxs,35000.0,,26.06,10.0,,7141778.0,,,39.99,29.0,150.70000000000002,118.0,1.0,65.0,
sigma,7993.556188734649,,4.391939870545795,3.6106637311002365,,59070.915654918244,,,7.587668224192549,0.6941679229284182,25.285366766770505,11.685190365910659,0.3866995896078875,6.947732922546696,
zeros,0,,0,14248,,0,,,270,139459,1562,0,133971,11,
missing,0,0,0,5804,0,4,0,0,0,29,193,29,0,29,0
0,5000.0,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0.0,26.0,verified
1,2500.0,60 months,15.27,0.0,RENT,30000.0,car,GA,1.0,0.0,9.4,4.0,1.0,12.0,verified
2,2400.0,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0.0,10.0,not verified


In [20]:
# independent variables

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

In [21]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [22]:
print(classification)

False


In [23]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [24]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok, val   

In [25]:
ok,val=check_y(y,df)

In [26]:
print(val)

enum


In [27]:
print(ok)

True


In [28]:
if val=='enum':
    print(df[y].levels())

[['not verified', 'verified']]


In [29]:
df.describe()

Rows:163987
Cols:15




,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
type,int,enum,real,real,enum,real,enum,enum,real,real,real,real,int,real,enum
mins,500.0,,5.42,0.0,,1896.0,,,0.0,0.0,0.0,1.0,0.0,0.0,
mean,13074.169141456336,,13.715904065566173,5.6843529329953295,,71915.67051974901,,,15.881530121290117,0.22735700606252815,54.07917280242254,24.579733834274638,0.1830388994249544,14.854273655448353,
maxs,35000.0,,26.06,10.0,,7141778.0,,,39.99,29.0,150.70000000000002,118.0,1.0,65.0,
sigma,7993.556188734649,,4.391939870545795,3.5461915319989035,,59070.19521200387,,,7.587668224192549,0.694106540371824,25.270482836105227,11.684157090192695,0.3866995896078875,6.947118561676316,
zeros,0,,0,14248,,0,,,270,139459,1562,0,133971,11,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,5000.0,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0.0,26.0,verified
1,2500.0,60 months,15.27,0.0,RENT,30000.0,car,GA,1.0,0.0,9.4,4.0,1.0,12.0,verified
2,2400.0,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0.0,10.0,not verified


In [30]:
allV=get_variables_types(df)
allV

{'addr_state': 'enum',
 'annual_inc': 'real',
 'bad_loan': 'int',
 'delinq_2yrs': 'int',
 'dti': 'real',
 'emp_length': 'int',
 'home_ownership': 'enum',
 'int_rate': 'real',
 'loan_amnt': 'int',
 'longest_credit_length': 'int',
 'purpose': 'enum',
 'revol_util': 'real',
 'term': 'enum',
 'total_acc': 'int',
 'verification_status': 'enum'}

In [31]:
meta_data['variables']=allV

In [32]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.9])

In [33]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [34]:
model_start_time = time.time()

In [35]:
aml.train(x=X,y=y,training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [37]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [38]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_0_AutoML_20181003_00...,0.785823,0.522657,0.396698,0.421877,0.177980
1,StackedEnsemble_BestOfFamily_0_AutoML_20181003...,0.784996,0.523250,0.391956,0.422185,0.178241
2,GBM_grid_0_AutoML_20181003_003247_model_1,0.784544,0.518154,0.417497,0.421158,0.177374
3,GBM_grid_0_AutoML_20181003_003247_model_2,0.783022,0.519539,0.392275,0.421796,0.177912
4,GBM_grid_0_AutoML_20181003_003247_model_0,0.782517,0.519939,0.407686,0.421937,0.178031
5,GBM_grid_0_AutoML_20181003_003247_model_4,0.780564,0.521617,0.421100,0.422932,0.178872
6,GBM_grid_0_AutoML_20181003_003247_model_3,0.780374,0.522223,0.421792,0.423088,0.179004
7,XRT_0_AutoML_20181003_003247,0.775741,0.528193,0.417997,0.424820,0.180472
8,DRF_0_AutoML_20181003_003247,0.774102,0.537229,0.415757,0.425434,0.180994
9,DeepLearning_0_AutoML_20181003_003247,0.752195,0.548348,0.468879,0.434282,0.188600


In [39]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [40]:
mod_best._id

'StackedEnsemble_AllModels_0_AutoML_20181003_003247'

In [41]:
# Get stacked ensemble  
se=get_stacked_ensemble(model_set)

In [42]:
print(se)

StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247


In [43]:
if se is not None:
  mod_best=h2o.get_model(se)

dir(mod_best)

confusion_matrix(metrics=None, thresholds=None)[source]
Get the confusion matrix for the specified metric

Parameters:	
metrics – A string (or list of strings) in {“min_per_class_accuracy”, “absolute_mcc”, “tnr”, “fnr”, “fpr”, “tpr”, “precision”, “accuracy”, “f0point5”, “f2”, “f1”,”mean_per_class_accuracy”}
thresholds – A value (or list of values) between 0 and 1

http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/metrics.html

In [44]:
mod_best.confusion_matrix()

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4491082665876067: 


,not verified,verified,Error,Rate
not verified,28157.0,14484.0,0.3397,(14484.0/42641.0)
verified,7543.0,68050.0,0.0998,(7543.0/75593.0)
Total,35700.0,82534.0,0.1863,(22027.0/118234.0)


In [45]:
mod_best.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4491082665876067: 


,not verified,verified,Error,Rate
not verified,28157.0,14484.0,0.3397,(14484.0/42641.0)
verified,7543.0,68050.0,0.0998,(7543.0/75593.0)
Total,35700.0,82534.0,0.1863,(22027.0/118234.0)


Confusion Matrix (Act/Pred) for max f2 @ threshold = 0.34540682761612657: 


,not verified,verified,Error,Rate
not verified,14624.0,28017.0,0.657,(28017.0/42641.0)
verified,1280.0,74313.0,0.0169,(1280.0/75593.0)
Total,15904.0,102330.0,0.2478,(29297.0/118234.0)


Confusion Matrix (Act/Pred) for max f0point5 @ threshold = 0.598529971801974: 


...

Confusion Matrix (Act/Pred) for max min_per_class_accuracy @ threshold = 0.5313532697534705: 


,not verified,verified,Error,Rate
not verified,34652.0,7989.0,0.1874,(7989.0/42641.0)
verified,14059.0,61534.0,0.186,(14059.0/75593.0)
Total,48711.0,69523.0,0.1865,(22048.0/118234.0)


Confusion Matrix (Act/Pred) for max mean_per_class_accuracy @ threshold = 0.5732548117680268: 


,not verified,verified,Error,Rate
not verified,36861.0,5780.0,0.1356,(5780.0/42641.0)
verified,17413.0,58180.0,0.2304,(17413.0/75593.0)
Total,54274.0,63960.0,0.1962,(23193.0/118234.0)


[, , , , , , , , , ]

In [46]:
mod_best._id

'StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247'

In [47]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics>

In [48]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [49]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.13283243041888848
RMSE: 0.36446183671118226
LogLoss: 0.41502682890195663
Null degrees of freedom: 118233
Residual degrees of freedom: 118229
Null deviance: 154600.60735250154
Residual deviance: 98140.56417678788
AIC: 98150.56417678788
AUC: 0.9019159548320826
Gini: 0.8038319096641653
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4491082665876067: 


,not verified,verified,Error,Rate
not verified,28157.0,14484.0,0.3397,(14484.0/42641.0)
verified,7543.0,68050.0,0.0998,(7543.0/75593.0)
Total,35700.0,82534.0,0.1863,(22027.0/118234.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4491083,0.8607006,252.0
max f2,0.3454068,0.9181200,312.0
max f0point5,0.5985300,0.8788335,174.0
max accuracy,0.4848849,0.8182418,232.0
max precision,0.9462183,1.0,0.0
max recall,0.1961826,1.0,379.0
max specificity,0.9462183,1.0,0.0
max absolute_mcc,0.5512065,0.6117913,198.0
max min_per_class_accuracy,0.5313533,0.8126451,209.0
max mean_per_class_accuracy,0.5732548,0.8170488,187.0


Gains/Lift Table: Avg response rate: 63.94 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3846509,0.7893550,299.0
max f2,0.1583682,0.8987170,390.0
max f0point5,0.7083000,0.8024969,129.0
max accuracy,0.5005176,0.7126292,232.0
max precision,0.9468332,0.9971182,0.0
max recall,0.1087959,1.0,396.0
max specificity,0.9468332,0.9999531,0.0
max absolute_mcc,0.7307278,0.4389645,120.0
max min_per_class_accuracy,0.5600324,0.7083324,199.0
max mean_per_class_accuracy,0.6777614,0.7250033,142.0


Gains/Lift Table: Avg response rate: 63.94 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100056,0.9458622,1.5614424,1.5614424,0.9983094,0.9983094,0.0156231,0.0156231,56.1442350,56.1442350
,2,0.0200027,0.9448854,1.5442378,1.5528437,0.9873096,0.9928118,0.0154379,0.0310611,54.4237807,55.2843716
,3,0.0299998,0.9440429,1.5587936,1.5548265,0.9966159,0.9940795,0.0155835,0.0466445,55.8793604,55.4826453
,4,0.0400054,0.9432213,1.5535095,1.5544971,0.9932375,0.9938689,0.0155438,0.0621883,55.3509536,55.4497084
,5,0.0500025,0.9423361,1.5482076,1.5532396,0.9898477,0.9930650,0.0154776,0.0776659,54.8207570,55.3239607
,6,0.1000051,0.9376606,1.5339266,1.5435831,0.9807172,0.9868911,0.0767002,0.1543661,53.3926629,54.3583118
,7,0.1499992,0.9286545,1.5140761,1.5337485,0.9680257,0.9806033,0.0756948,0.2300610,51.4076070,53.3748545
,8,0.2000017,0.9142339,1.4706965,1.5179849,0.9402909,0.9705248,0.0735386,0.3035995,47.0696470,51.7984859
,9,0.2999983,0.8725402,1.3717343,1.4692360,0.8770194,0.9393572,0.1371688,0.4407683,37.1734262,46.9236035
,10,0.4000034,0.7353691,1.2062674,1.4034911,0.7712280,0.8973231,0.1206329,0.5614012,20.6267414,40.3491099



<bound method ModelBase.coef_norm of >


In [50]:
bm=stackedensemble_df(aml_leaderboard_df)

In [51]:
bm

['GBM_grid_0_AutoML_20181003_003247_model_1',
 'GLM_grid_0_AutoML_20181003_003247_model_0',
 'DRF_0_AutoML_20181003_003247',
 'XRT_0_AutoML_20181003_003247',
 'DeepLearning_0_AutoML_20181003_003247']

In [52]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_0_AutoML_20181003_00...,0.785823,0.522657,0.396698,0.421877,0.177980
1,StackedEnsemble_BestOfFamily_0_AutoML_20181003...,0.784996,0.523250,0.391956,0.422185,0.178241
2,GBM_grid_0_AutoML_20181003_003247_model_1,0.784544,0.518154,0.417497,0.421158,0.177374
3,GBM_grid_0_AutoML_20181003_003247_model_2,0.783022,0.519539,0.392275,0.421796,0.177912
4,GBM_grid_0_AutoML_20181003_003247_model_0,0.782517,0.519939,0.407686,0.421937,0.178031
5,GBM_grid_0_AutoML_20181003_003247_model_4,0.780564,0.521617,0.421100,0.422932,0.178872
6,GBM_grid_0_AutoML_20181003_003247_model_3,0.780374,0.522223,0.421792,0.423088,0.179004
7,XRT_0_AutoML_20181003_003247,0.775741,0.528193,0.417997,0.424820,0.180472
8,DRF_0_AutoML_20181003_003247,0.774102,0.537229,0.415757,0.425434,0.180994
9,DeepLearning_0_AutoML_20181003_003247,0.752195,0.548348,0.468879,0.434282,0.188600


In [53]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [54]:
bm

['GBM_grid_0_AutoML_20181003_003247_model_1',
 'XRT_0_AutoML_20181003_003247',
 'DRF_0_AutoML_20181003_003247',
 'DeepLearning_0_AutoML_20181003_003247',
 'GLM_grid_0_AutoML_20181003_003247_model_0',
 'StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247']

In [55]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]


In [56]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
1,StackedEnsemble_BestOfFamily_0_AutoML_20181003...,0.784996,0.523250,0.391956,0.422185,0.178241
2,GBM_grid_0_AutoML_20181003_003247_model_1,0.784544,0.518154,0.417497,0.421158,0.177374
7,XRT_0_AutoML_20181003_003247,0.775741,0.528193,0.417997,0.424820,0.180472
8,DRF_0_AutoML_20181003_003247,0.774102,0.537229,0.415757,0.425434,0.180994
9,DeepLearning_0_AutoML_20181003_003247,0.752195,0.548348,0.468879,0.434282,0.188600
10,GLM_grid_0_AutoML_20181003_003247_model_0,0.748344,0.551138,0.462745,0.436204,0.190274


In [57]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [58]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247


In [59]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247
stackedensemble


In [60]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [61]:
meta_data['models']=bm

In [62]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass    

In [63]:
print(models_path)

/Users/bear/Downloads/AutoML/hunAq3K6I/models


In [64]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass        

{'algo': 'gbm', 'model_id': 'GBM_grid_0_AutoML_20181003_003247_model_1', 'varimp': [('loan_amnt', 13790.80078125, 1.0, 0.36337776396370103), ('term', 6952.900390625, 0.5041694460613322, 0.18320396596858468), ('addr_state', 3470.388916015625, 0.25164520690734454, 0.09144227259817375), ('int_rate', 3460.663330078125, 0.25093998419462704, 0.09118601034573005), ('dti', 2710.388916015625, 0.19653600679234487, 0.07141681468655704), ('emp_length', 2214.00244140625, 0.16054197841915113, 0.058337385140259886), ('purpose', 1386.7762451171875, 0.10055806527222488, 0.03654056490713698), ('annual_inc', 1144.5565185546875, 0.08299420292626, 0.030158247881294013), ('revol_util', 985.2030029296875, 0.07143914400309309, 0.025959396407324812), ('longest_credit_length', 813.091796875, 0.05895899808664347, 0.021424388890264632), ('total_acc', 520.2718505859375, 0.03772600727387057, 0.01370879216645741), ('home_ownership', 358.5807189941406, 0.026001442895300732, 0.009448346179124241), ('delinq_2yrs', 78.7

In [65]:
# DeepLearning

mod,mod_id=get_model_by_algo("Dee",best_models)


In [66]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass       

{'algo': 'deeplearning', 'model_id': 'DeepLearning_0_AutoML_20181003_003247', 'varimp': [('loan_amnt', 1.0, 1.0, 0.057139774459626), ('annual_inc', 0.6492429971694946, 0.6492429971694946, 0.03709759842775653), ('term.36 months', 0.6443209052085876, 0.6443209052085876, 0.03681635120324076), ('term.60 months', 0.6017407178878784, 0.6017407178878784, 0.03438332890328681), ('int_rate', 0.5695430040359497, 0.5695430040359497, 0.03254355879567203), ('dti', 0.5403347015380859, 0.5403347015380859, 0.030874602978595563), ('purpose.small_business', 0.5323701500892639, 0.5323701500892639, 0.030419510305137783), ('revol_util', 0.4409337639808655, 0.4409337639808655, 0.025194855825500618), ('home_ownership.RENT', 0.39914199709892273, 0.39914199709892273, 0.02280688369159714), ('home_ownership.MORTGAGE', 0.276227205991745, 0.276227205991745, 0.01578356024998096), ('purpose.educational', 0.2624789774417877, 0.2624789774417877, 0.014997989571417012), ('addr_state.NH', 0.25939443707466125, 0.2593944370

In [67]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass     

{'algo': 'drf', 'model_id': 'DRF_0_AutoML_20181003_003247', 'varimp': [('loan_amnt', 123539.453125, 1.0, 0.2228840883972794), ('addr_state', 85879.015625, 0.6951545716986919, 0.15493889300826416), ('int_rate', 53601.1484375, 0.43387878998675145, 0.09670467858111173), ('dti', 46907.97265625, 0.37970034243868195, 0.08462916468858045), ('term', 41309.95703125, 0.33438675650807403, 0.07452948739642512), ('revol_util', 33860.30859375, 0.274084980443368, 0.06108918100950623), ('total_acc', 31307.00390625, 0.2534170511065228, 0.056482628420204094), ('annual_inc', 31131.365234375, 0.2519953298067103, 0.05616574936434039), ('longest_credit_length', 29828.21484375, 0.2414468745751136, 0.053814666536046454), ('purpose', 28012.44140625, 0.2267489510246284, 0.05053873324416365), ('emp_length', 27478.240234375, 0.2224248168443153, 0.04957495253927705), ('home_ownership', 9887.966796875, 0.08003893935705003, 0.017839406034881225), ('delinq_2yrs', 6830.953125, 0.05529369729432336, 0.012324085315560377

In [68]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass     

{'algo': 'drf', 'model_id': 'XRT_0_AutoML_20181003_003247', 'varimp': [('loan_amnt', 107676.0546875, 1.0, 0.19883127109738963), ('addr_state', 100249.28125, 0.9310266942909995, 0.1851172210514802), ('term', 50597.24609375, 0.46990248891078457, 0.09343130916195833), ('int_rate', 48523.203125, 0.45064061146951556, 0.08960144558658868), ('dti', 43028.5, 0.39961066668794964, 0.07945509680164033), ('annual_inc', 33411.66796875, 0.3102980329815959, 0.061696952316750445), ('revol_util', 30078.9453125, 0.27934665139613285, 0.05554284977389249), ('purpose', 28430.447265625, 0.2640368589668011, 0.05249878428493126), ('total_acc', 26800.810546875, 0.24890223387787516, 0.049489547540917674), ('emp_length', 25706.701171875, 0.2387411133002746, 0.04746919902069951), ('longest_credit_length', 25396.5234375, 0.2358604567302024, 0.04689643441327701), ('home_ownership', 10680.3427734375, 0.09918958123451165, 0.019721990516475735), ('delinq_2yrs', 6400.908203125, 0.05944597637517336, 0.011819719044301114

In [69]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass     

{'algo': 'glm', 'model_id': 'GLM_grid_0_AutoML_20181003_003247_model_0', 'coef': {'Intercept': -1.643191526405633, 'addr_state.AK': 0.031111965796962005, 'addr_state.AL': -0.013296057771368757, 'addr_state.AR': 0.13150827846679108, 'addr_state.AZ': 0.06252148904002464, 'addr_state.CA': 0.1517902825591284, 'addr_state.CO': 0.03668192794759818, 'addr_state.CT': -0.10173028414432689, 'addr_state.DC': 0.1291408602740501, 'addr_state.DE': 0.13527748755041608, 'addr_state.FL': 0.03948988937950739, 'addr_state.GA': 0.13294014640982532, 'addr_state.HI': 0.020993943093315547, 'addr_state.IA': -0.21073258759220903, 'addr_state.ID': -0.20360371996444435, 'addr_state.IL': -0.04386217401577293, 'addr_state.IN': 0.05001305521098949, 'addr_state.KS': 0.06762418787443246, 'addr_state.KY': 0.03342361221740867, 'addr_state.LA': 0.007074231672246331, 'addr_state.MA': -0.04170193279874198, 'addr_state.MD': 0.056561327567320865, 'addr_state.ME': -0.09490831917704058, 'addr_state.MI': -0.03782350685439326, 

In [70]:
predictions_df=predictions_test(mod_best,test,run_id)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [71]:
predictions_df.head()

predict,not verified,verified
verified,0.437667,0.562333
not verified,0.7105,0.2895
not verified,0.70557,0.29443
verified,0.553836,0.446164
verified,0.62137,0.37863
verified,0.383057,0.616943
verified,0.407588,0.592412
verified,0.368503,0.631497
verified,0.535562,0.464438
verified,0.594965,0.405035


In [72]:
predictions_df.describe()

Rows:16354
Cols:3




,predict,not verified,verified
type,enum,real,real
mins,,0.05296612913498899,0.07791647516368842
mean,,0.360804076078991,0.6391959239210091
maxs,,0.9220835248363116,0.947033870865011
sigma,,0.23287204602157835,0.2328720460215783
zeros,,0,0
missing,0,0,0
0,verified,0.43766718748348854,0.5623328125165115
1,not verified,0.7105002344288622,0.2894997655711378
2,not verified,0.705569730301578,0.29443026969842195


In [73]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)    


In [74]:
meta_data

{'X': ['loan_amnt',
  'emp_length',
  'delinq_2yrs',
  'total_acc',
  'bad_loan',
  'longest_credit_length',
  'term',
  'home_ownership',
  'purpose',
  'addr_state',
  'int_rate',
  'annual_inc',
  'dti',
  'revol_util'],
 'analysis': 0,
 'balance': False,
 'balance_threshold': 0.2,
 'classification': False,
 'end_time': 1538541557.772541,
 'execution_time': 394.14366126060486,
 'max_models': 9,
 'min_mem_size': 6,
 'mod_best': 'StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247',
 'mod_best_algo': 'stackedensemble',
 'model_execution_time': 385.7865219116211,
 'models': ['GBM_grid_0_AutoML_20181003_003247_model_1',
  'XRT_0_AutoML_20181003_003247',
  'DRF_0_AutoML_20181003_003247',
  'DeepLearning_0_AutoML_20181003_003247',
  'GLM_grid_0_AutoML_20181003_003247_model_0',
  'StackedEnsemble_BestOfFamily_0_AutoML_20181003_003247'],
 'nthreads': 1,
 'project': None,
 'run_id': 'hunAq3K6I',
 'run_time': 333,
 'scale': False,
 'start_time': 1538541163.6288798,
 'target': None,
 'variab

In [75]:
# Clean up
os.chdir(server_path)

In [76]:
h2o.cluster().shutdown()

H2O session _sid_a408 closed.


**When availble for your models you will need to make the following plots: **

## Variable Importance Plots

The Variable Importance Plot graphs the VIP values for each X variable. 

https://www.researchgate.net/figure/Random-forest-variable-importance-plot-Variables-are-ranked-in-terms-of-importance-on_fig2_295097543



## Partial Dependence Plots

For models that include only numerical values, you can view a Partial Dependence Plot (PDP) for that model. This provides a graphical representation of the marginal effect of a variable on the class probability (classification) or response (regression).

https://www.kaggle.com/dansbecker/partial-dependence-plots



## Gains/Lift Charts

[Score: Gains/Lift Table](http://h2o-release.s3.amazonaws.com/h2o/master/1648/docs-website/userguide/scoregainslift.html)  

The Gains/Lift Table page uses predicted data to evaluate model performance. The accuracy of the classification model for a random sample is evaluated according to the results when the model is and is not used.

The Gains/Lift Table is particularly useful for direct marketing applications, for example. The gains/lift chart shows the effectiveness of the current model(s) compared to a baseline, allowing users to quickly identify the most useful model.

To create a Gains/Lift table, H2O applies the model to each entry in the original dataset to find the response probability (Pi^), then orders the entries according to their predicted response probabilities. Finally, H2O divides the dataset into equal groups and calculates the average response rate for each group.

H2O uses the response rate of the top ten groups to evaluate the model performance; the highest response and greatest variation rates indicate the best model.

The lift is calculated from the gains. H2O uses the following formula to calculate the lift: λk=rkr⎯⎯⎯

where λk is the lift for k, rk is the response rate for k, and r⎯⎯⎯ is the average response rate for k. In other words, λk defines how much more likely k customers are to respond in comparison to the average response rate.

Requirements:

The vactual column must contain actual binary class labels.
The vpredict column must contain probabilities.

Evaluating Classifiers: Gains and Lift Charts https://youtu.be/1dYOcDaDJLY 

Understanding And Interpreting Gain And Lift Charts

   https://www.datasciencecentral.com/profiles/blogs/understanding-and-interpreting-gain-and-lift-charts
   
   

**Binomial Classification Metrics**

When you train a binomial model, there are some additional metrics we get, the most commonly used of which is AUC, which normally ranges from 0.5 to 1.0, higher being better. AUC stands for Area Under Curve. 

See

* ROC Curves and Area Under the Curve (AUC) Explained https://youtu.be/OAl6eAyP-yo   


* ROC Curves https://youtu.be/21Igj5Pr6u4  



## Confusion Matrix

A confusion matrix, also known as an error matrix, is a specific table layout that allows visualization of the performance of an algorithm, typically a supervised learning one (in unsupervised learning it is usually called a matching matrix). Each row of the matrix represents the instances in a predicted class while each column represents the instances in an actual class.

https://en.wikipedia.org/wiki/Confusion_matrix


## Max F1 and Max F2

By default H2O AI uses max F1 for binary classification in the confusion matrix.  For 

See https://en.wikipedia.org/wiki/F1_score
    
For binary classification also generate the max F2 confusion matrix.  

